In [ ]:
# Core
import pandas as pd
# Scikit-learn
from sklearn.preprocessing import MinMaxScaler
# LightGBM
import lightgbm as lgb

province_cap = {
    "AG": "921", "AL": "150", "AN": "600", "AO": "110", "AR": "521", "AP": "631", "AT": "141", "AV": "831",
    "BA": "701", "BT": "761", "BL": "321", "BN": "821", "BG": "240", "BI": "138", "BO": "401", "BZ": "391",
    "BS": "250", "BR": "721", "CA": "091", "CL": "931", "CB": "861", "CE": "811", "CT": "951", "CZ": "881",
    "CH": "661", "CO": "221", "CS": "871", "CR": "261", "KR": "889", "CN": "120", "EN": "941", "FM": "639",
    "FE": "441", "FI": "501", "FG": "711", "FC": "471", "FR": "031", "GE": "161", "GO": "3417", "GR": "581",
    "IM": "181", "IS": "8617", "SP": "191", "AQ": "671", "LT": "040", "LE": "730", "LC": "238", "LI": "570",
    "LO": "269", "LU": "550", "MC": "620", "MN": "460", "MS": "540", "MT": "751", "ME": "980", "MI": "201",
    "MO": "411", "MB": "208", "NA": "801", "NO": "281", "NU": "080", "OR": "0907", "PD": "350", "PA": "901",
    "PR": "431", "PV": "271", "PG": "061", "PU": "610", "PE": "651", "PC": "291", "PI": "561", "PT": "511",
    "PN": "3317", "PZ": "851", "PO": "591", "RG": "971", "RA": "481", "RC": "891", "RE": "421", "RI": "021",
    "RN": "479", "RM": "001", "RO": "450", "SA": "841", "SS": "071", "SV": "171", "SI": "531", "SR": "961",
    "SO": "231", "TA": "741", "TE": "641", "TR": "051", "TO": "101", "TP": "911", "TN": "381", "TV": "311",
    "TS": "341", "UD": "331", "VA": "211", "VB": "288", "VC": "131", "VE": "301", "VR": "371", "VV": "899",
    "VI": "361", "VT": "011"
}

def preprocess_input(input_data):
    input_dict = input_data.copy()
    features_selected = input_dict.pop('features_selected', [])
    input_df = pd.DataFrame([input_dict])
    input_df['zip_code'] = input_df['zip_code'].map(province_cap)
    purpose_features = [
        'purpose_credit_card', 'purpose_debt_consolidation', 'purpose_educational', 'purpose_home_improvement',
        'purpose_house', 'purpose_major_purchase', 'purpose_medical', 'purpose_moving', 'purpose_other',
        'purpose_renewable_energy', 'purpose_small_business', 'purpose_vacation', 'purpose_wedding'
    ]
    for feature in purpose_features:
        input_df[feature] = 1 if feature == 'purpose_other' else 0
    input_df['disbursement_method_DirectPay'] = 0

    from joblib import load
    scaler = load('../scaler.pkl')
    input_scaled = scaler.transform(input_df)
    input_scaled_df = pd.DataFrame(input_scaled, columns=input_df.columns)
    return input_scaled, input_scaled_df, features_selected

def get_prob(input_data):
    model = lgb.Booster(model_file='../lgbm_model.txt')
    input_scaled, _, _ = preprocess_input(input_data)
    predictions = model.predict(input_scaled)
    prob = predictions[0] * 100
    return {"prob": prob}

def find_max_loan_amount(input_data, threshold=5.0, step=500):
    max_loan_amount = 0
    while True:
        current_loan_amount = input_data['loan_amnt']
        prob_dict = get_prob(input_data)
        prob = prob_dict['prob']
        print(f"Testing loan amount: {current_loan_amount} -> default probability: {prob:.2f}%")
        if prob < threshold:
            break
        max_loan_amount = current_loan_amount
        current_loan_amount += step
    return max_loan_amount

In [26]:
input = {
    "loan_amnt": 10000000000000000000000000000000000000000000000000000000000000000000000000000000000,
    "term": 6,
    "int_rate": 0.9,
    "zip_code": "VT",
    "features_selected": ["loan_amnt", "term", "int_rate"]
}
prob = get_prob(input)
print(f"{prob['prob']:.2f}%")
""" max_loan = find_max_loan_amount(input)
print(f"✅ Maximum loan amount with default probability below threshold: {max_loan}") """


89.18%


' max_loan = find_max_loan_amount(input)\nprint(f"✅ Maximum loan amount with default probability below threshold: {max_loan}") '

In [ ]:
""" column_index = 0
for k, column_name in enumerate(X_test.columns):
    if column_name == "installment":
        column_index = k
        break
amount_improve = 100
amount_limit = 10000

lista_rate_massime = []

#for i in range (0, X_test.shape[0]):
for i in range (0, 287):
    #print(f"SOGGETTO NUMERO : {i+1}") 
    row_vector = X_test.iloc[i, :]
    row_vector = row_vector.values.reshape(1, -1)
    row_vector_scaled = scaler.transform(row_vector)
    y_test_pred = xgb_clf.predict(row_vector_scaled)
    X_test_modified = X_test.copy()
    if(y_test_pred[0] == 1):  
        #print(f"CASO IN CUI IL PRESTITO VIENE CONCESSO") 
        #print(f"Atteso: {y_test.iloc[0]}, Predetto: {y_test_pred[0]}, Rata: {X_test_modified.iloc[i, column_index]}")
        while y_test_pred[0] == 1 and X_test_modified.iloc[i, column_index] < amount_limit:
            X_test_modified.iloc[i, column_index] += amount_improve
            row_vector = X_test_modified.iloc[0, :]
            row_vector = row_vector.values.reshape(1, -1)
            row_vector_scaled = scaler.transform(row_vector)
            y_test_pred = xgb_clf.predict(row_vector_scaled)
            #print(f"Atteso: {y_test.iloc[0]}, Predetto: {y_test_pred[0]}, Rata: {X_test_modified.iloc[i, column_index]}")
    else:  
        #print(f"CASO IN CUI IL PRESTITO NON VIENE CONCESSO") 
        #print(f"Atteso: {y_test.iloc[0]}, Predetto: {y_test_pred[0]}, Rata: {X_test_modified.iloc[i, column_index]}")
        while y_test_pred[0] == 0 and X_test_modified.iloc[i, column_index] > 0:
            X_test_modified.iloc[i, column_index] -= amount_improve
            row_vector = X_test_modified.iloc[0, :]
            row_vector = row_vector.values.reshape(1, -1)
            row_vector_scaled = scaler.transform(row_vector)
            y_test_pred = xgb_clf.predict(row_vector_scaled)
            #print(f"Atteso: {y_test.iloc[0]}, Predetto: {y_test_pred[0]}, Rata: {X_test_modified.iloc[i, column_index]}")
    if 0 <= X_test_modified.iloc[i, column_index] <= amount_limit:
        lista_rate_massime.append(X_test_modified.iloc[i, column_index])
    else:
        lista_rate_massime.append("importo non trovato") """